# **Open Energi SDK**

This SDK can be used to interact with the Open Energi API. 

- An **Entity Code** is the unique identifier that Open Energi assigns to an asset e.g. `l1234`.
- Time based filters (start / end) can be in any format e.g.:
  - Python `datetime.datetime`
  - Pandas `pd.Timestamp`
  - Strings `YYYY-MM-DD HH:mm:ss.SSS` (with or without time-zones, `Z` suffix, etc.)
- The term **metric** usually refers to **default/active profiles** e.g. `cd-power-target` (see below).
- The term **variable** usually refers to **raw/resampled readings** e.g. `active-power` (see below).

In [1]:
import os

username = os.environ['OE_USERNAME']
password = os.environ['OE_PASSWORD']

entity_code = "l4662"

In [2]:
import datetime
import json

import pandas as pd

import oesdk

pd.options.mode.chained_assignment = None

# **Entity API**

This can be used for a given entity code to describe:
- the entity details
- the **asset hierarchy**

In [3]:
entityApi = oesdk.entity.EntityApi(username, password)

## Describe the details for a given entity code

In [4]:
e_dict = entityApi.entityDetailsAsDict(entity_code)

print("These are the retrieved entity details:\n", json.dumps(e_dict, indent=4))

These are the retrieved entity details:
 {
    "code": "l4662",
    "type": "L",
    "name": "SDK dummy load",
    "asset_parent": "d993",
    "service_parent": null,
    "has_asset_children": false,
    "has_service_children": false,
    "updated_at": "2019-12-06T10:56:04.0869095Z",
    "updated_by": "bot@openenergi.com",
    "tags": [
        {
            "key": "em-mode",
            "value": "cd-mode",
            "updated_at": "2019-12-06T13:39:04.880447Z",
            "updated_by": "sdk-api-sample-user@openenergi.com",
            "inherited": false
        }
    ]
}


In [5]:
assert e_dict['code'] == 'l4662', 'The expected entity code is not present'

## Describe an entity and its ancestors

In [6]:
eh_dict = entityApi.explainHierarchy(entity_code)

print("This is the hierarchy breakdown from the entity code:\n", json.dumps(eh_dict, indent=4))

This is the hierarchy breakdown from the entity code:
 {
    "EntityName": "SDK dummy load",
    "EntityIpAddress": "",
    "ParentEntityCode": "d993",
    "ParentName": "SDK dummy device",
    "GrandParentEntityCode": "s535",
    "GrandParentName": "SDK dummy site"
}


In [7]:
assert eh_dict['EntityName'] == 'SDK dummy load', 'The expected entity hierarchy is not present'

# **Demand profiles API**

There are 2 different type of profiles:

- **active** profiles belong to a specific date `YYYY-MM-DD`
- **default** profiles belong to a specific ISO Week Day ID (Monday=1 to Sunday=7)

For a given day the **active** profile has **higher priority** than the **default** profile. 

In [8]:
demandApi = oesdk.demand_profiles.DemandApi(username, password)

## Retrieve an active profile

In [9]:
target_date_str = datetime.datetime(2019, 12, 1).strftime("%Y-%m-%d")
retrieved_ap_df = demandApi.getActiveProfile(entity_code, target_date_str)

retrieved_ap_df.head()

,HalfhourStart,profile_id,entity_code,target_date,cd-power-target,Date,Timestamp,SettlementPeriod
0,1,4937,l4662,2019-12-01,1292.466654,2019-12-01,2019-12-01 00:00:00,1
1,2,4937,l4662,2019-12-01,1290.711337,2019-12-01,2019-12-01 00:30:00,2
2,3,4937,l4662,2019-12-01,1143.750920,2019-12-01,2019-12-01 01:00:00,3
3,4,4937,l4662,2019-12-01,1152.255688,2019-12-01,2019-12-01 01:30:00,4
4,5,4937,l4662,2019-12-01,1220.345264,2019-12-01,2019-12-01 02:00:00,5


In [10]:
assert len(retrieved_ap_df) >= 0 and len(retrieved_ap_df) <= 48, 'The active profile does not have the expected length'

## Upsert an active profile

The dataframes:

- must have these columns: `Timestamp`, `HalfhourStart`
- they can have extra columns for the metrics e.g. `cd-power-target`

In [11]:
ap_df_to_upsert = retrieved_ap_df[['Timestamp', 'HalfhourStart', 'cd-power-target']]
ap_df_to_upsert.head()

,Timestamp,HalfhourStart,cd-power-target
0,2019-12-01 00:00:00,1,1292.466654
1,2019-12-01 00:30:00,2,1290.711337
2,2019-12-01 01:00:00,3,1143.750920
3,2019-12-01 01:30:00,4,1152.255688
4,2019-12-01 02:00:00,5,1220.345264


In [12]:
upsertedActiveProfileId = demandApi.upsertActiveProfile(
    entityCode=entity_code, 
    inDf=ap_df_to_upsert,
    targetDateStr=ap_df_to_upsert.iloc[0]['Timestamp'].strftime("%Y-%m-%d") # '2019-12-01'
)
print("The profile ID for the updated ACTIVE profile is:", upsertedActiveProfileId)

The profile ID for the updated ACTIVE profile is: 4937


### Upsert profiles across multiple days

We can upsert **active** profiles across multiple days. 
The Pandas dataframe can contain extra slices of data according to the columns `Timestamp` and `HalfhourStart`.

In [13]:
listOfProfileIDs = demandApi.upsertActiveProfiles(
    entityCode=entity_code, 
    inDf=ap_df_to_upsert,
)
print("The list of ACTIVE profile IDs is:", listOfProfileIDs)

The list of ACTIVE profile IDs is: [4937]


## Retrieve a default profile

In [14]:
iso_weekday_id = datetime.datetime(2019, 12, 1).isoweekday() # this should be "7"
print("On '2019-12-01' the ISO week day ID is a Sunday:", iso_weekday_id)

On '2019-12-01' the ISO week day ID is a Sunday: 7


In [15]:
retrieved_dp_df = demandApi.getDefaultProfile(entity_code, iso_weekday_id)
retrieved_dp_df.head()

,HalfhourStart,default_profile_id,profile_id,entity_code,week_day_id,cd-power-target
0,1,569,4938,l4662,7,1292.466654
1,2,569,4938,l4662,7,1290.711337
2,3,569,4938,l4662,7,1143.750920
3,4,569,4938,l4662,7,1152.255688
4,5,569,4938,l4662,7,1220.345264


In [16]:
assert len(retrieved_dp_df) >= 0 and len(retrieved_dp_df) <= 48, 'The default profile does not have the expected length'

## Upsert a default profile

In [17]:
dp_df_to_upsert=retrieved_dp_df[['HalfhourStart', 'cd-power-target']]
dp_df_to_upsert.head()

,HalfhourStart,cd-power-target
0,1,1292.466654
1,2,1290.711337
2,3,1143.750920
3,4,1152.255688
4,5,1220.345264


In [18]:
upsertedDefaultProfileId = demandApi.upsertDefaultProfile(
    entityCode=entity_code, 
    inDf=dp_df_to_upsert, 
    weekDayId=iso_weekday_id
)
print("The profile ID for the updated DEFAULT profile is:", upsertedDefaultProfileId)

The profile ID for the updated DEFAULT profile is: 569


# **Historical API**

This can be used to retrieve readings for a given asset.

Readings can be:

- resampled
- raw

In [19]:
historicalApi = oesdk.historical_timeseries.HistoricalApi(username, password)

power_variable = 'active-power'

## Raw readings

In [20]:
start_raw_readings = '2019-12-01 10:00:00'
end_raw_readings = pd.Timestamp('2019-12-01 12:00:00')

raw_readings_df = historicalApi.getRawReadings(
    start_raw_readings,
    end_raw_readings,
    variable=power_variable,
    entity_code=entity_code
)

print("Retrieved {} raw readings".format(len(raw_readings_df)))

Retrieved 3 raw readings


In [21]:
raw_readings_df.head()

,EntityCode,Time,Type,active-power
time,,,,
2019-12-01 10:04:32.345000+00:00,l4662,2019-12-01 10:04:32.345000+00:00,raw,1
2019-12-01 11:18:24.002000+00:00,l4662,2019-12-01 11:18:24.002000+00:00,raw,5
2019-12-01 11:47:23.496000+00:00,l4662,2019-12-01 11:47:23.496000+00:00,raw,10


In [22]:
assert len(raw_readings_df) == 3, 'The raw readings do not have the expected length'

## Resampled readings

In [23]:
start_resampled = '2019-12-01 15' # this works as well
end_resampled = '2019-12-01 16:01' # this works as well

resampled_readings_df = historicalApi.getResampledReadings(
    start_resampled,
    end_resampled,
    variable=power_variable,
    entity_code=entity_code
)

print("Retrieved {} resampled readings".format(len(resampled_readings_df)))

Retrieved 3 resampled readings


In [24]:
resampled_readings_df.head()

,Time,active-power,EntityCode,Type
0,2019-12-01T16:00:00Z,100,l4662,30m-compliance
1,2019-12-01T15:30:00Z,50,l4662,30m-compliance
2,2019-12-01T15:00:00Z,10,l4662,30m-compliance


In [25]:
assert len(resampled_readings_df) == 3, 'The resampled readings do not have the expected length'